## Imports

In [1]:
# Math, Optimization and Stats. Packages
import numpy as np                         
from scipy import interpolate           
import torch as torch  
from scipy.optimize import minimize
import emcee
import time
import corner

# Plotting packages and settings
import matplotlib.pyplot as plt  
from matplotlib import colors
from matplotlib import rc
from matplotlib.pyplot import figure
rc('font',**{'family':'sans-serif','sans-serif':['DejaVu Sans'],'size':12})
rc('mathtext',**{'default':'regular'})
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'retina'")
# from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,
#                                                   mark_inset)

# Data Management
import pandas as pd
from pickle import dump, load  

# Paralellization Packages and settings
import dask
from multiprocessing import Pool
import multiprocessing as mp
import os
mp.set_start_method('fork')
os.environ["OMP_NUM_THREADS"] = "1" 

## Experimental Data

In [2]:
print("Loading experimental data...")
input_dict = load(open('exp_data/experimental_data_NEW.p', 'rb'))
xs_exp = torch.tensor(input_dict['exp_params'])
sqs_exp = torch.tensor(input_dict['sqs'])
rdfs_exp = torch.tensor(input_dict['rdfs'])
print("Success!!!")
print(xs_exp)

Loading experimental data...
Success!!!
tensor([[1.8502, 0.8571],
        [1.8502, 0.7965],
        [1.8502, 0.7438],
        [1.8502, 0.6977],
        [1.8925, 0.8571],
        [1.8925, 0.7965],
        [1.8925, 0.7438],
        [1.8925, 0.6977],
        [1.9330, 0.8571],
        [1.9330, 0.7965],
        [1.9330, 0.7438],
        [1.9330, 0.6977],
        [1.9718, 0.8571],
        [1.9718, 0.7965],
        [1.9718, 0.7438],
        [1.9718, 0.6977],
        [1.7291, 0.8010],
        [1.8614, 0.9102],
        [1.9428, 1.0521]], dtype=torch.float64)


## Training Data

In [3]:
print("Loading training data...")
input_dict = load(open('training_data/train_data.p', 'rb'))
xd = torch.tensor(input_dict['xs'])
sqs = torch.tensor(input_dict['sqs'])
rdfs = torch.tensor(input_dict['rdfs'])
print("Success!!!")

Loading training data...
Success!!!


## GP Model

In [4]:
def se_kernel(x1, x2, l, width):
    K = width**2 * torch.exp(-(torch.cdist(x1/l,x2/l,p=2)**2)/2)
    return K

def local_surrogate(Xi, Xd, l, width, y, KddInv, μd):
    V     = torch.stack([((n/(n-6))*((n/6)**((6)/(n-6))))*e*((s/r_pr)**n - (s/r_pr)**6) for n,s,e in zip(Xi[:,0],Xi[:,1],Xi[:,2])])
    pmf_μ = torch.exp(-V/kbT)
    μ    = torch.zeros([len(Xi), qnum])
    for i in range (len(pmf_μ)):
        μ[i]  = rdf2sq2(r_pr, pmf_μ[i], q, ρ) + 1    
    Kid = se_kernel(Xi, Xd, l, width)
    return (μ +(Kid @ KddInv) @ (y-μd)).T

def local_surrogate1(Xi, Xd, l, width, y, KddInv):   
    Kid = se_kernel(Xi, Xd, l, width)
    return (1 +(Kid @ KddInv) @ (y-1)).T

def rdf2sq2(r, rdf, q, ρ):
    import numpy as np
    dr = r[1] - r[0] 
    sq   = torch.zeros(len(q))
    for j in range (len(q)):
        sq[j] = (4*np.pi*ρ*np.trapz(r*(rdf-1)*np.sin(q[j]*r),dx = dr)/q[j])
    return sq   

In [5]:
N_particles = 1_000
name = "A"
timestep = 0.0025 # 48.#### fs
kbT = 1
ρ = 0.1
nsims = 960

rmax = (0.95/2)*((N_particles/ρ)**(1/3))
rmin = 0
rnum = 250
r = np.linspace(rmin,rmax,rnum)

#assume atom size of ~2.5 A^-1
qmax = np.pi/(2.5*(rmax - rmin)/rnum) #qmax in reduced space is (π/Δr)*1/2.5
qmin = 1.25                           #qmin in reduced space is qmin for standard diffractometers * 2.5 A
qnum = 236                            #qnum selected so that the spacing is approximately 0.05A^{-1} for a 2.5 A particle
q = torch.linspace(qmin,qmax,qnum) 
print('qmin =', qmin)
print('qmax =', qmax)
print('Δq =', q[1]- q[0])

#Limit the training set to liquid-like region
Xd = torch.tensor(xd).float()
y = torch.tensor(sqs).float()

n = len(Xd)
η = len(q)

index = torch.arange(0,len(Xd),1)

#from hyperparameter training and model validation
arr = [3.38224890e+00, 1.88830504e-01, 1.16976918e-01, 1.37565712e-01,1.37362647e-06]
l = torch.tensor([arr[0],arr[1],arr[2]]).float()
w = torch.tensor(arr[3]).float()
σn = torch.tensor(arr[4]).float()

qmin = 1.25
qmax = 30.698908169815425
Δq = tensor(0.1253)


/scratch/local/u1262705/1073679/ipykernel_4028722/197537984.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xd = torch.tensor(xd).float()
/scratch/local/u1262705/1073679/ipykernel_4028722/197537984.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(sqs).float()


## Prior Functions

In [6]:
μ_n = 3
σ_n = 1
prior_n_dist = torch.distributions.normal.Normal(μ_n, σ_n)
def log_prior_n(n):
    return prior_n_dist.log_prob(n)
    
μ_σ = 2
σ_σ = 1
prior_σ_dist = torch.distributions.normal.Normal(μ_σ, σ_σ)
def log_prior_σ(σ):
    return prior_σ_dist.log_prob(σ)

μ_ϵ = 0.7
σ_ϵ = 1.5
prior_ϵ_dist = torch.distributions.normal.Normal(μ_ϵ, σ_ϵ)
def log_prior_ϵ(ϵ):
    return prior_ϵ_dist.log_prob(ϵ)

μ_σn = 0
σ_σn = 1.5
prior_σn_dist = torch.distributions.normal.Normal(μ_σn, σ_σn)
def log_prior_σn(σn):
    return prior_σn_dist.log_prob(σn)
    
def log_prior(pos):
    return log_prior_n(pos[0]) + log_prior_σ(pos[1]) + log_prior_ϵ(pos[2]) + log_prior_σn(pos[3])

## Prior Distributions

In [7]:
# Define training set data
Xd = xd.float()
y = sqs.float()

# Compute subset GP prediction over prior predictive set
Kdd = se_kernel(Xd,Xd,l,w) + torch.eye(len(Xd))*σn
KddInv = torch.linalg.inv(Kdd)

In [8]:
# exp_index = 15
# sq_exp = sqs_exp[exp_index]
# mie_exp = xs_exp[exp_index]
# print("Exp Params:",xs_exp[exp_index])

In [9]:
# replicates = 3
# noise = np.random.normal(0,0.003,(replicates,len(sq_exp)))
# pos = torch.tensor([12.0500,  1.9167,  0.1833, 0.009])
# surr = local_surrogate1(pos[:3].unsqueeze(dim=0),Xd,l, w, y, KddInv).squeeze(dim=1)
# for n in noise:
#     plt.plot(q,sq_exp+n)
# plt.plot(q, surr,linestyle='dashed')
# plt.show()

# Posterior Functions

In [10]:
# input_dict = load(open('training_data/hyperparameter/hyperParams_laplace.p', 'rb'))
# μ_hp = input_dict['mean']
# cov_hp = input_dict['cov']

In [11]:
def log_posterior(pos):
    prior_θ = log_prior(pos)
    pos = pos.exp()
    pos[0] += 6
    ssq_noisless = (local_surrogate1(pos[:3].unsqueeze(dim=0),Xd,l, w, y, KddInv).squeeze(dim=1) - sq_exp).repeat(int(replicates),1)
    ssq = torch.sum((ssq_noisless - noise)**2)
    LH = - ssq/(2*((pos[3])**(2))) - replicates*qnum*torch.log(pos[3]) - (replicates*qnum/2)*np.log(2*np.pi)
    return prior_θ + LH

In [12]:
# def scipy_log_posterior(pos):
#     posTensor = torch.tensor(pos).float()
#     out = -log_posterior(posTensor)
#     return out 

def log_posterior_emcee(pos):
    posTensor = torch.tensor(pos).float()
    out = log_posterior(posTensor)
    return out 

# bnds_n = ((None, None),(None, None),(None, None),(None,None))
# out = minimize(scipy_log_posterior, (np.log(12), np.log(1.5), np.log(0.3), np.log(0.0001)),method='Nelder-Mead', options={'disp': True},bounds=bnds_n)

## Posterior Sampling

In [ ]:
noise_arr = [0.005, 0.0075, 0.01, 0.02, 0.05, 0.1]

for i in range (len(sqs_exp)):
    for j in range (len(noise_arr)):
        sq_exp = sqs_exp[i]
        mie_exp = xs_exp[i]
        print("Exp Params:",xs_exp[i])

        replicates = 4
        noise = np.random.normal(0,noise_arr[j],(replicates,len(sq_exp)))

        # Create an initial position for the MCMC walkers
        ndim, nwalkers = 4, 160 # Make nwalkers = mp.cpu_count * 2 
        p0 = np.ones((nwalkers, ndim))

        # Set up the backend
        # Don't forget to clear it in case the file already exists
        filename = "results/reactor_NEW/exp_" + str(i) + "unc_" + str(j) + ".h5"

        p0[:,0] = np.random.normal(μ_n, σ_n, nwalkers)
        p0[:,1] = np.random.normal(μ_σ, σ_σ, nwalkers)
        p0[:,2] = np.random.normal(μ_ϵ, σ_ϵ, nwalkers)
        p0[:,3] = np.random.normal(μ_σn, σ_σn, nwalkers)

        backend = emcee.backends.HDFBackend(filename)
        backend.reset(nwalkers, ndim)

        # Use pool to parallelize calculation
        print('Starting MCMC Run...')
        with Pool() as pool:
            #Create a sampler to run MCMC

            sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior_emcee, pool = pool, backend = backend)

            max_n = 200000

            # We'll track how the average autocorrelation time estimate changes
            index = 0
            autocorr = np.empty(max_n)

            # This will be useful to testing convergence
            old_tau = np.inf

            # Now we'll sample for up to max_n steps
            for sample in sampler.sample(p0, iterations=max_n, progress=True, store=True, tune=True):
                # Only check convergence every 100 steps
                if sampler.iteration % 1000:
                    continue

                # Compute the autocorrelation time so far
                # Using tol=0 means that we'll always get an estimate even
                # if it isn't trustworthy
                tau = sampler.get_autocorr_time(tol=0)
                autocorr[index] = np.mean(tau)
                index += 1

                # Check convergence
                converged = np.all(tau * 100 < sampler.iteration)
                converged &= np.all(np.abs(old_tau - tau) / tau < 0.01)
                if converged:
                    break
                old_tau = tau

Exp Params: tensor([1.8502, 0.8571], dtype=torch.float64)
Starting MCMC Run...


  8%|▊         | 15168/200000 [31:30<5:58:05,  8.60it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 13%|█▎        | 25934/200000 [53:18<5:34:28,  8.67it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 20%|██        | 40972/200000 [1:24:52<5:08:39,  8.59it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
